<a href="https://colab.research.google.com/github/swilsonmfc/computational/blob/master/ProbabilisticDataStructures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probabilistic Data Structures

![](https://previews.123rf.com/images/toonartist/toonartist1406/toonartist140600186/29370854-yes-no-maybe-concept-with-three-red-dice-on-a-white-background-.jpg)

# Install

In [1]:
!pip install pdsa

  Created wheel for pdsa: filename=pdsa-0.5.0-cp36-cp36m-linux_x86_64.whl size=2613669 sha256=42184c602baac4228e0f028bf77d42255ac911f676510484bec8822867b25c4c
  Stored in directory: /root/.cache/pip/wheels/7e/45/12/58fe9be197ae164c9cf17615ffea5944bdbf71ae28ebb9eae1
Successfully built pdsa


In [2]:
!pip install murmurhash3

  Created wheel for murmurhash3: filename=murmurhash3-2.3.5-cp36-cp36m-linux_x86_64.whl size=25330 sha256=57dd68d7dd5f637e8c611d98ab793adde152ed21618a81537f64189aff5beca3
  Stored in directory: /root/.cache/pip/wheels/06/eb/e4/f57324cd9c1bf001c9ba6d6926ad5231eca80964ed80b3610e
Successfully built murmurhash3


# Setup

In [3]:
import sys
import random
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import mmh3

from pdsa.membership.bloom_filter import BloomFilter
from pdsa.cardinality.hyperloglog import HyperLogLog
from pdsa.frequency.count_min_sketch import CountMinSketch

# Sizing Objects

## GetSizeOf
* getsizeof tells us the memory taken up by an object
* For complex objects it doesn't recursively gather sizes

In [4]:
obj = "123"
sys.getsizeof(obj)

52

In [5]:
class Simple():
  pass

simple = Simple()
sys.getsizeof(simple)

56

In [6]:
simple.text = 'Holding a string'
sys.getsizeof(simple)

56

## Recursive GetSizeOf


In [7]:
def getsizeof_r(obj, seen=None):
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    
    obj_id = id(obj)
    # Exit recursion if we've seen this object before
    if obj_id in seen:
        return 0
    
    # Take care of newly visited objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([getsizeof_r(v, seen) for v in obj.values()])
        size += sum([getsizeof_r(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += getsizeof_r(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([getsizeof_r(i, seen) for i in obj])
    return size

In [8]:
simple = Simple()
simple.text = 'Holding a string'
getsizeof_r(simple)

286

# Dict

## Memory
* Dicts allocate memory in steps
* Dicts don't report recursive memory usage

In [9]:
d = {}
for letter in 'abcdefghijklmnopqrstuvwxyz':
  d[letter] = letter
  print(f'Dictionary Size: {len(d)}, sys.getsizeof(d) = {sys.getsizeof(d)}')

Dictionary Size: 1, sys.getsizeof(d) = 240
Dictionary Size: 2, sys.getsizeof(d) = 240
Dictionary Size: 3, sys.getsizeof(d) = 240
Dictionary Size: 4, sys.getsizeof(d) = 240
Dictionary Size: 5, sys.getsizeof(d) = 240
Dictionary Size: 6, sys.getsizeof(d) = 368
Dictionary Size: 7, sys.getsizeof(d) = 368
Dictionary Size: 8, sys.getsizeof(d) = 368
Dictionary Size: 9, sys.getsizeof(d) = 368
Dictionary Size: 10, sys.getsizeof(d) = 368
Dictionary Size: 11, sys.getsizeof(d) = 648
Dictionary Size: 12, sys.getsizeof(d) = 648
Dictionary Size: 13, sys.getsizeof(d) = 648
Dictionary Size: 14, sys.getsizeof(d) = 648
Dictionary Size: 15, sys.getsizeof(d) = 648
Dictionary Size: 16, sys.getsizeof(d) = 648
Dictionary Size: 17, sys.getsizeof(d) = 648
Dictionary Size: 18, sys.getsizeof(d) = 648
Dictionary Size: 19, sys.getsizeof(d) = 648
Dictionary Size: 20, sys.getsizeof(d) = 648
Dictionary Size: 21, sys.getsizeof(d) = 648
Dictionary Size: 22, sys.getsizeof(d) = 1184
Dictionary Size: 23, sys.getsizeof(d) = 

In [10]:
d = {}
for letter in 'abcdefghijklmnopqrstuvwxyz':
  d[letter] = letter
  print(f'Dictionary Size: {len(d)}, sys.getsizeof(d) = {sys.getsizeof(d)}, getsizeof_r(d) = {getsizeof_r(d)}')

Dictionary Size: 1, sys.getsizeof(d) = 240, getsizeof_r(d) = 298
Dictionary Size: 2, sys.getsizeof(d) = 240, getsizeof_r(d) = 356
Dictionary Size: 3, sys.getsizeof(d) = 240, getsizeof_r(d) = 414
Dictionary Size: 4, sys.getsizeof(d) = 240, getsizeof_r(d) = 472
Dictionary Size: 5, sys.getsizeof(d) = 240, getsizeof_r(d) = 530
Dictionary Size: 6, sys.getsizeof(d) = 368, getsizeof_r(d) = 716
Dictionary Size: 7, sys.getsizeof(d) = 368, getsizeof_r(d) = 774
Dictionary Size: 8, sys.getsizeof(d) = 368, getsizeof_r(d) = 832
Dictionary Size: 9, sys.getsizeof(d) = 368, getsizeof_r(d) = 890
Dictionary Size: 10, sys.getsizeof(d) = 368, getsizeof_r(d) = 948
Dictionary Size: 11, sys.getsizeof(d) = 648, getsizeof_r(d) = 1286
Dictionary Size: 12, sys.getsizeof(d) = 648, getsizeof_r(d) = 1344
Dictionary Size: 13, sys.getsizeof(d) = 648, getsizeof_r(d) = 1402
Dictionary Size: 14, sys.getsizeof(d) = 648, getsizeof_r(d) = 1460
Dictionary Size: 15, sys.getsizeof(d) = 648, getsizeof_r(d) = 1518
Dictionary Siz

# BloomFilter
* BloomFilters help identify members in a set.
* They answer the question probabilisticly:
  * They can tell you definitively no it is not a member. 
  * They can tell you possibly it is a member.
  * Can have false positives but not false negatives
* Examples:
  * Have I seen this user before?
  * Is this a new flight in a radar tracking system?
* Works by hashing the value to a smaller space
  * If the hash exists, it could be in the set
  * It can't tell you definitely, because there could be hash collisions
  * If the hash isn't in the bloom filter, you definitively know it's not a member
* Key decisions:
  * How much space should the filter offer?
  * What likelhood of false positive can you tolerate?
  * Need deletes?  Look at Cuckoo Filter

## Internals
* https://www.interviewcake.com/concept/java/bloom-filter


### Initialize
![](https://www.interviewcake.com/images/svgs/python__bloom_filter__overview.svg?bust=209)

### Addition
![](https://www.interviewcake.com/images/svgs/python__bloom_filter__add_a_new_word.svg?bust=209)

### Testing

![](https://www.interviewcake.com/images/svgs/python__bloom_filter__check_a_word_in_the_set.svg?bust=209)

## Implementation
* Trivial Implementation of a Filter

### Tools - Hashing & Binary

In [11]:
for i in range(8):
  print(bin(i))

0b0
0b1
0b10
0b11
0b100
0b101
0b110
0b111


In [12]:

print(f'Hashing Hello:    {mmh3.hash("hello")}')
print(f'Bin Hashed Hello: {bin(mmh3.hash("hello"))}')

Hashing Hello:    613153351
Bin Hashed Hello: 0b100100100010111111101001000111


### Hashing to Fixed Length

In [13]:
length = 8
bloom = [0 for x in range(length)]
print(bloom)

[0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
hashed = mmh3.hash("hello")
position = hashed % length
bloom[position] = 1
print(bloom)

[0, 0, 0, 0, 0, 0, 0, 1]


### Testing Presence

In [15]:
test = mmh3.hash("world")
position = test % length
print(f'Hashed value {test}, position {position}')
if bloom[position] == 1:
  print('Possibly Present')
else:
  print('Definitively Not Present')

Hashed value -74040069, position 3
Definitively Not Present


In [16]:
test = mmh3.hash("ninja")
position = test % length
print(f'Hashed value {test}, position {position}')
if bloom[position] == 1:
  print('Possibly Present')
else:
  print('Definitively Not Present')

Hashed value -531200449, position 7
Possibly Present


### Extensions
* Multiple hashes (use mmh3 with seed)
* Increase filter length (more space)

## Examples

In [17]:
bf = BloomFilter(100000, 5) #100,000 Positions, 5 Hashes
print(bf)
bf.add('hello')

print('Test Hello', bf.test('hello'))
print('Test World', bf.test('world'))
print('Test Ninja', bf.test('ninja'))

<BloomFilter (length: 100000, hashes: 5)>
Test Hello True
Test World False
Test Ninja False


In [18]:
# Initial State
bf = BloomFilter(100000, 5)
d  = {}
print(f'Size BloomFilter {getsizeof_r(bf)}')
print(f'Size Dict {getsizeof_r(d)}')

Size BloomFilter 296
Size Dict 240


In [19]:
# Insert 10000
letters = string.ascii_lowercase
for i in range(10000):
  value = ''.join(random.choice(letters) for i in range(5))
  bf.add(value)
  d[value] = 1
print(f'Size BloomFilter {getsizeof_r(bf):,}')
print(f'Size Dict {getsizeof_r(d):,}')

Size BloomFilter 296
Size Dict 834,928


In [20]:
# Collisions
tp = 0
fp = 0
tn = 0
fn = 0
for i in range(100000):
  value = ''.join(random.choice(letters) for i in range(5))
  present_bf = bf.test(value)
  present_d  = value in d

  if present_bf and present_d:
    tp += 1
  elif present_bf and not present_d:
    fp += 1
  elif not present_bf and present_d:
    fn += 1
  else:
    tn += 1
  
print(f'True  Positives {tp:,}')
print(f'False Positives {fp:,}')
print(f'True  Negatives {tn:,}')
print(f'False Negatives {fn:,}')

True  Positives 85
False Positives 924
True  Negatives 98,991
False Negatives 0


## Considerations
* Advantages
  * Bloom filters take up O(1) space 
  * Insert and lookup operations are both O(1)
  * Bloom filters can identify true negatives. 
* Disadvantages
  * Accuracy goes down as more elements are added
  * Cannot identify true positives
  * Cannot iterate through the items in the filter
  * Cannot delete items.

# HyperLogLog
* Answers how many distinct items are there in a list?
* HyperLogLog answers this question probabilistically
* An exact count requires memory proportional to the cardinality (think dict)


## Internals
* http://blog.notdot.net/2012/09/Dam-Cool-Algorithms-Cardinality-Estimation
* https://engineering.fb.com/2018/12/13/data-infrastructure/hyperloglog/
* https://towardsdatascience.com/big-data-with-sketchy-structures-part-2-hyperloglog-and-bloom-filters-73b1c4a2e6ad

![](https://miro.medium.com/max/571/1*U0GjwE-zxNs5zkzQY0p4XA.png)

## Implementation

In [21]:
def trailing_zeros(num):
  """Trivial implementation to count the number of trailing zeros"""
  return len(str(num)) - len(str(num).rstrip('0'))

### Splitting hash

In [22]:
length = 3
hash_length = 32

In [23]:
hashed = mmh3.hash("hello")

hashed_bin = bin(hashed)
bucket_bin = bin(hashed)[-length :]
zeros_bin  = bin(hashed)[: -length]
bucket_num = int(bucket_bin, 2)
zeros_num  = int(zeros_bin, 2)

print(f'Hashed {hashed}')
print(f'Hashed Binary  {hashed_bin}')
print(f'Bucket Binary  {bucket_bin:>32}')
print(f'Zeros  Binary  {zeros_bin}')
print(f'Bucket Number  {bucket_num}')
print(f'Zeros  Number  {zeros_num}')
print(f'Trailing Zeros {trailing_zeros(zeros_num)}')

Hashed 613153351
Hashed Binary  0b100100100010111111101001000111
Bucket Binary                               111
Zeros  Binary  0b100100100010111111101001000
Bucket Number  7
Zeros  Number  76644168
Trailing Zeros 0


### Estimating Cardinality

In [24]:
def cardinality(length):
  d = {}
  buckets = [0 for x in range(2 ** length)]

  letters = string.ascii_lowercase
  for i in range(10000):
    value = ''.join(random.choice(letters) for i in range(3))
    d[value] = 1

    hashed = mmh3.hash(value)
    hashed_bin = bin(hashed)
    bucket_bin = bin(hashed)[-length :]
    zeros_bin  = bin(hashed)[: -length]
    bucket_num = int(bucket_bin, 2)
    zeros_num  = int(zeros_bin, 2)
    zeros      = trailing_zeros(zeros_bin)

    if zeros > buckets[bucket_num]:
      buckets[bucket_num] = zeros

  print(f'Buckets {2 ** length}') 
  print(f'Actual Cardinality    : {len(d):,}')

  num_buckets = len(buckets)
  print(f'Estimated Cardinality : {2 ** (float(sum(buckets)) / num_buckets) * num_buckets * 0.79402:,.0f}')

### Comparing Precision

In [25]:
# With 8 (2 ** 3) Buckets there's higher levels of error
cardinality(3)

Buckets 8
Actual Cardinality    : 7,584
Estimated Cardinality : 5,016


In [26]:
# With larger number of buckets 256 (2 ** 8) the estimate is more accurate
cardinality(8)

Buckets 256
Actual Cardinality    : 7,634
Estimated Cardinality : 7,714


## Examples

In [27]:
hll = HyperLogLog(10)
hll.add('Hello')
hll.count()

1

In [28]:
# Initial State
hll = HyperLogLog(10)
d   = {}
print(f'Size HyperLogLog {getsizeof_r(hll)}')
print(f'Size Dict {getsizeof_r(d)}')

Size HyperLogLog 72
Size Dict 240


In [29]:
# Insert 10000
letters = string.ascii_lowercase
for i in range(10000):
  value = ''.join(random.choice(letters) for i in range(5))
  hll.add(value)
  d[value] = 1

print(f'Unique Items HyperLogLog {hll.count():,}')
print(f'Unique Items Dict        {len(d):,}')

Unique Items HyperLogLog 9,874
Unique Items Dict        9,993


In [30]:
# Memory
print(f'Size HyperLogLog {getsizeof_r(hll):,}')
print(f'Size Dict {getsizeof_r(d):,}')

Size HyperLogLog 72
Size Dict 834,658


In [31]:
precision_min, precision_max, items = 4, 16, 10000
hlls = []
d    = {}

for x in range(precision_min, precision_max):
  hlls.append( HyperLogLog(x) )

letters = string.ascii_lowercase
for i in range(items):
  value = ''.join(random.choice(letters) for i in range(5))
  d[value] = 1

  for hll in hlls:
    hll.add(value)

print(f'Actual Items {len(d):,}')
for x, hll in zip( range(precision_min, precision_max), hlls ):
  diff     = len(d) - hll.count()
  diff_per = diff / items * 100. 
  print(f'HyperLogLog Precision {x:<2}  Length {len(hll):<6} Size {getsizeof_r(hll)}  Unique {hll.count():<6} ({diff:<6} {diff_per:.2f}%)')

Actual Items 9,993
HyperLogLog Precision 4   Length 16     Size 72  Unique 5824   (4169   41.69%)
HyperLogLog Precision 5   Length 32     Size 72  Unique 9232   (761    7.61%)
HyperLogLog Precision 6   Length 64     Size 72  Unique 9960   (33     0.33%)
HyperLogLog Precision 7   Length 128    Size 72  Unique 9052   (941    9.41%)
HyperLogLog Precision 8   Length 256    Size 72  Unique 10086  (-93    -0.93%)
HyperLogLog Precision 9   Length 512    Size 72  Unique 9353   (640    6.40%)
HyperLogLog Precision 10  Length 1024   Size 72  Unique 10185  (-192   -1.92%)
HyperLogLog Precision 11  Length 2048   Size 72  Unique 10062  (-69    -0.69%)
HyperLogLog Precision 12  Length 4096   Size 72  Unique 9739   (254    2.54%)
HyperLogLog Precision 13  Length 8192   Size 72  Unique 10003  (-10    -0.10%)
HyperLogLog Precision 14  Length 16384  Size 72  Unique 9971   (22     0.22%)
HyperLogLog Precision 15  Length 32768  Size 72  Unique 9901   (92     0.92%)


## Considerations
* Advantages
  * O(log(log(n)) complexity for space
  * O(1) for insertion
  * Higher precision improves accuracy (at expense of space)
* Disadvantages
  * A poor hashing function can distort
  * The algorithm can suffer from variance

# CountMinSketch
* What is the frequency of items appearing in a stream?
* Quality is driven by the number of cells in the structure
* Examples:
  * View counts of website pages
  * Songs played 
  * Not best structure for tracking ad impressions (need accurate counts)
* Complexity
  * Dictionaries grow at O(n) complexity
  * O(1) Space (Memory = d * w = number of hashes * number of counters)
  * O(1) Insertion 
* Variations exist to reduce bias

## Internals

![](https://www.waitingforcode.com/public/images/articles/count_min_sketch.png)

## Implementation

### Dictionary

In [32]:
items = 50000
hashes = 5
width  = 20

animals = ['dog', 'cat', 'mouse', 'bird', 'tiger', 'lion', 'otter', 'rat', 'toad', 'elephant',
           'eagle', 'beaver', 'yak', 'fish', 'rabbit', 'cow', 'goat', 'monkey', 'shark', 'turtle']
parade = [random.choice(animals) for i in range(items)]

d = {}
for animal in parade:
  d.setdefault(animal, 0)
  d[animal] = d[animal] + 1
print(f'Size of {getsizeof_r(d)}')

Size of 2280


In [33]:
d

{'beaver': 2479,
 'bird': 2557,
 'cat': 2483,
 'cow': 2484,
 'dog': 2424,
 'eagle': 2566,
 'elephant': 2536,
 'fish': 2501,
 'goat': 2489,
 'lion': 2447,
 'monkey': 2554,
 'mouse': 2555,
 'otter': 2499,
 'rabbit': 2471,
 'rat': 2415,
 'shark': 2483,
 'tiger': 2540,
 'toad': 2470,
 'turtle': 2503,
 'yak': 2544}

### Hashing

In [34]:
sketch = np.repeat(0, hashes * width)
getsizeof_r(sketch)

for animal in parade:
  for h in range(hashes):
    hashed = mmh3.hash(animal, seed=h)
    index = hashed % width
    sketch[h * width + index] = sketch[h * width + index] + 1
print(f'Size of {getsizeof_r(sketch)}')

Size of 960


### Estimating Counts

In [35]:
results = {}
for animal in animals:
  min = items
  for h in range(hashes):
    hashed = mmh3.hash(animal, seed=h)
    index = hashed % width
    value = sketch[h * width + index]
    if value < min:
      min = value
  results[animal] = min
results

{'beaver': 2479,
 'bird': 5040,
 'cat': 4954,
 'cow': 2484,
 'dog': 2424,
 'eagle': 5065,
 'elephant': 2536,
 'fish': 4985,
 'goat': 2489,
 'lion': 2447,
 'monkey': 2554,
 'mouse': 2555,
 'otter': 2499,
 'rabbit': 2471,
 'rat': 2415,
 'shark': 2483,
 'tiger': 2540,
 'toad': 2470,
 'turtle': 2503,
 'yak': 2544}

## Examples

In [36]:
cms = CountMinSketch(5, 20)
d = {}

for animal in parade:
  d.setdefault(animal, 0)
  d[animal] = d[animal] + 1
  cms.add(animal)

for animal in animals:
  print(animal, cms.frequency(animal))

dog 2424
cat 4953
mouse 2555
bird 2557
tiger 7488
lion 7488
otter 2499
rat 4904
toad 4953
elephant 2536
eagle 2566
beaver 2479
yak 2544
fish 7488
rabbit 2471
cow 2484
goat 4904
monkey 2554
shark 2483
turtle 2503


In [37]:
cms = CountMinSketch(5, 100)
d = {}

for animal in parade:
  d.setdefault(animal, 0)
  d[animal] = d[animal] + 1
  cms.add(animal)

for animal in animals:
  print(animal, cms.frequency(animal))

dog 2424
cat 2483
mouse 2555
bird 2557
tiger 5041
lion 2447
otter 2499
rat 2415
toad 2470
elephant 2536
eagle 2566
beaver 2479
yak 2544
fish 5041
rabbit 2471
cow 2484
goat 2489
monkey 2554
shark 2483
turtle 2503


## Considerations
* Advantages
  * Similar to Bloom Filters
  * By design, they can never undercount
  * Parallelizable
* Disadvantages
  * Biased estimator
  * They are prone to overcount when not sized
  * There are variations to counter bias